# <u> LSTM ENCODER DECODER

# Data

In [1]:
import utils
import numpy as np

#utils.setup_nltk()
PRESIDENT = 'obama'
speeches = utils.read_all_text_files(PRESIDENT)

In [2]:
assert np.mean([len(x) for x in speeches]) == 4878.9

### Preprocessing

In [3]:
filter_list = [':', '(', ')', ',', '-',]
filtered_speeches = []

for speech in speeches:
    filtered_speech = []
    for word in speech:
        # filter out unwanted words
        if word not in filter_list:
            # lower word
            filtered_speech.append(word.lower())
    filtered_speeches.append(filtered_speech)

### Create N-Grams

In [4]:
from nltk import ngrams

WINDOW = 10
grams = [ngrams(s, WINDOW+1) for s in filtered_speeches]
flat_grams = [ng for speech in grams for ng in speech]

### Split Data

In [5]:
import pandas as pd

X = [' '.join(list(x[0:WINDOW])) for x in flat_grams]
Y = [x[-1] for x in flat_grams]
df = pd.DataFrame.from_dict({'x':X, 'y':Y})

# persist
csv_name = '../data/' + PRESIDENT + '_preproc/encdec_' + str(WINDOW) + 'grams.csv'
df.to_csv(csv_name, index=False)

In [6]:
#assert len(df) == 230020

In [7]:
import torchtext

XFIELD = torchtext.data.Field(sequential=True)
YFIELD = torchtext.data.Field(sequential=True)
DATA = torchtext.data.TabularDataset(csv_name,'csv', 
                                     [('x', XFIELD),('y', YFIELD)], skip_header=True)

XFIELD.build_vocab(DATA)  
YFIELD.build_vocab(DATA)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes f

In [8]:
from torchtext.data import BucketIterator, Iterator
import torch

BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator = Iterator(DATA, BATCH_SIZE, device=device, train=True)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [9]:
assert device.type == 'cuda'

# Neural Networks

### Encoder

In [10]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers)

    def forward(self, x, h0, c0):
        x = self.embedding(x).unsqueeze(0)
        out, (h0, c0) = self.lstm(x, (h0, c0))
        return out, (h0, c0)

### Decoder

In [11]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, dropout=0.5)
        self.dense = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim = 1)
  
    def forward(self, x, h0, c0):
        x = self.embedding(x)
        x, (h0, c0) = self.lstm(x, (h0, c0))
        x = self.dense(x.squeeze(0))
        x = self.softmax(x)
        return x, (h0, c0)

### Training

In [12]:
HIDDEN_SIZE = 100
EMBEDDING_SIZE = 100
NUM_LAYERS = 2
ENC_LEARNING_RATE = 0.01
DEC_LEARNING_RATE = 0.01
criterion = nn.NLLLoss()

encoder = Encoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
decoder = Decoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
enc_optimizer = torch.optim.Adam(encoder.parameters(), lr = ENC_LEARNING_RATE)
dec_optimizer = torch.optim.Adam(decoder.parameters(), lr = DEC_LEARNING_RATE)

In [13]:
from tqdm import tqdm

EPOCHS = 10
for ep in range(EPOCHS):
    ep_loss = 0
    
    for batch in tqdm(train_iterator):
        if len(batch) != BATCH_SIZE: break;
        inp = batch.x
        target = batch.y
        
        # init
        loss = 0
        h0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        c0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        
        # encode
        for w in range(inp.size(0)):
            enc_out, (h0, c0) = encoder(inp[w], h0, c0)
            
        # decode
        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (h0, c0) = decoder(cur, h0, c0)
        cur = torch.argmax(dec_out,dim=1)
        
        # loss
        target_onehot = torch.nn.functional.one_hot(target.squeeze(), len(YFIELD.vocab))
        loss += criterion(dec_out, target.squeeze())
        
        # optimize
        ep_loss += loss
        loss.backward()
        enc_optimizer.step()
        dec_optimizer.step()
        
    print('AVG_LOSS={}, (ABS={})'.format(round((ep_loss/(len(DATA)/BATCH_SIZE)).item(),4), round(ep_loss.item(),2)))

  0%|          | 0/3589 [00:00<?, ?it/s]/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
  0%|          | 0/3589 [00:00<?, ?it/s]

AVG_LOSS=6.1346, (ABS=22014.24)


  0%|          | 0/3589 [00:00<?, ?it/s]

AVG_LOSS=5.6216, (ABS=20173.12)


  0%|          | 0/3589 [00:00<?, ?it/s]

...

AVG_LOSS=4.9961, (ABS=17928.69)


  0%|          | 0/3589 [00:00<?, ?it/s]

AVG_LOSS=4.9616, (ABS=17804.71)


100%|█████████▉| 3588/3589 [02:28<00:00, 24.15it/s]

AVG_LOSS=4.9345, (ABS=17707.43)


# Generate Text!

### reload models

In [14]:
pass

### predict

In [19]:
from torch import torch

def voc_index(words):
    return torch.tensor([XFIELD.vocab.stoi[x] for x in words]).to(device)

def predict(inp, RND_FACTOR=0):
    with torch.no_grad():

        h0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        c0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        
        for w in range(inp.size(0)):
                enc_out, (h0, c0) = encoder(inp[w], h0, c0)

        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (h0, c0) = decoder(cur, h0, c0)
        
        # randomize
        rnd = torch.rand(dec_out.shape).to(device) * RND_FACTOR
        cur = torch.argmax(dec_out.add(rnd),dim=1)
        
        # TODO pick more than first one
        return YFIELD.vocab.itos[cur[0].item()]

    
text = ['good', 'evening', 'ladies', 'and', 'gentlemen', 'i', 'am', 'very', 'happy', 'to']
for i in range(100):
    cur_window = text[-WINDOW:]
    vecs = voc_index(cur_window).view(WINDOW,1).repeat(1,BATCH_SIZE)
    text.append(predict(vecs, 1))
    
text

['good',
 'evening',
 'ladies',
 'and',
 'gentlemen',
 'i',
 'am',
 'very',
 'happy',
 'to',
 'the',
 'world',
 '.',
 'and',
 'i',
 '’',
 'm',
 'not',
 'a',
 'new',
 '.',
 'and',
 'i',
 '’',
 'm',
 'not',
 'a',
 'lot',
 'of',
 'the',
 'world',
 '.',
 'we',
 '’',
 're',
 'not',
 'a',
 'lot',
 'of',
 'the',
 'united',
 'states',
 '.',
 'applause',
 '.',
 'the',
 'united',
 'states',
 '.',
 'applause',
 '.',
 'the',
 'world',
 '’',
 's',
 'economy',
 '.',
 'applause',
 '.',
 'the',
 'united',
 'states',
 '.',
 'applause',
 '.',
 'and',
 'it',
 '’',
 's',
 'a',
 'lot',
 'of',
 'the',
 'world',
 '’',
 's',
 'security',
 '.',
 'applause',
 '.',
 'and',
 'i',
 '’',
 'm',
 'not',
 'a',
 'new',
 'and',
 'the',
 'united',
 'states',
 'and',
 'the',
 'world',
 '.',
 'applause',
 '.',
 'the',
 'united',
 'states',
 'and',
 'the',
 'world',
 '.',
 'and',
 'i',
 '’',
 'm',
 'not',
 'a']